In [2]:
# ------------------------------------------define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()

# this should import glob, os, and many other standard libs
from src.utils.Notebook_imports import *

# local imports
from src.utils.Utils_io import Console_and_file_logger, init_config
from src.visualization.Visualize import show_2D_or_3D

# import external libs
import cv2
import pandas as pd

EXPERIMENT = 'temp/MotionGenerator-tests'

# ------------------------------------------generator hyperparameters
SEED = 42 # define a seed for the generator shuffle
BATCHSIZE = 1 # 32, 64, 24, 16, 1 for 3D use: 4
GENERATOR_WORKER = BATCHSIZE # if not set, use batchsize
DIM = [40, 64, 64] # network input params for spacing of 3, (z,y,x)
SPACING = [4, 4, 4] # if resample, resample to this spacing, (z,y,x)
# maybe we can drop this channel?
#IMG_CHANNELS = 1 # Currently our model needs that image channel
MASK_VALUES = [1, 2, 3]  #channel order: Background, RV, MYO, LV
MASK_CLASSES = len(MASK_VALUES) # no of labels
BORDER_MODE = cv2.BORDER_REFLECT_101 # border mode for the data generation
IMG_INTERPOLATION = cv2.INTER_LINEAR # image interpolation in the genarator
MSK_INTERPOLATION = cv2.INTER_NEAREST # mask interpolation in the generator
AUGMENT = False # Not implemented for the AX2SAX case
SHUFFLE = True
RESAMPLE = True
SCALER = 'MinMax' # MinMax Standard or Robust

Console_and_file_logger(EXPERIMENT, logging.INFO)
config = init_config(config=locals(), save=True)
print(config)

search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/dynamic-cmr-models


2021-02-05 15:37:01,764 INFO -------------------- Start --------------------
2021-02-05 15:37:01,765 INFO Working directory: /mnt/ssd/git/dynamic-cmr-models.
2021-02-05 15:37:01,765 INFO Log file: ./logs/temp/MotionGenerator-tests.log
2021-02-05 15:37:01,765 INFO Log level for console: INFO


{'EXPERIMENT': 'temp/MotionGenerator-tests', 'SEED': 42, 'BATCHSIZE': 1, 'GENERATOR_WORKER': 1, 'DIM': [40, 64, 64], 'SPACING': [4, 4, 4], 'MASK_VALUES': [1, 2, 3], 'MASK_CLASSES': 3, 'BORDER_MODE': 4, 'IMG_INTERPOLATION': 1, 'MSK_INTERPOLATION': 0, 'AUGMENT': False, 'SHUFFLE': True, 'RESAMPLE': True, 'SCALER': 'MinMax', 'CONFIG_PATH': 'reports/config/temp/MotionGenerator-tests', 'TENSORBOARD_LOG_DIR': 'reports/tensorboard_logs/temp/MotionGenerator-tests', 'MODEL_PATH': 'models/temp/MotionGenerator-tests', 'HISTORY_PATH': 'reports/history/temp/MotionGenerator-tests'}


In [3]:
# load the dataframe
df = pd.read_csv('/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAx_3D_dicomTags_phase')
df['phase'] = pd.Categorical(df.phase, 
                      categories=['ED','MS','ES','PF','MD'],
                      ordered=True)
df.describe()

,background,rv,myo,lv,t,t_norm,x-axis,y-axis,z-axis,t-axis,...,LargestImagePixelValue,PatientBirthDate,YYYY,MM,DD,ED#,MS#,ES#,PF#,MD#
count,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.0,...,1380.000000,1.345000e+03,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000
mean,12202.307246,177.481159,82.334058,110.117391,11.303623,2.000000,244.369565,252.333333,14.094203,0.0,...,232.659420,1.986875e+07,2004.913043,6.557971,14.195652,15.557971,4.905797,9.394928,13.253623,18.405797
std,3946.906566,72.270755,26.995487,48.132996,6.902080,1.414726,60.244556,49.008196,3.205298,0.0,...,501.594311,1.327721e+05,11.030959,3.538902,9.639897,10.071188,1.719604,2.115204,2.425277,3.384128
min,3159.000000,35.000000,25.000000,21.000000,0.000000,0.000000,126.000000,156.000000,8.000000,0.0,...,0.000000,1.900010e+07,1900.000000,1.000000,1.000000,1.000000,2.000000,2.000000,3.000000,7.000000
25%,10516.500000,126.000000,62.750000,74.000000,5.000000,1.000000,256.000000,256.000000,12.000000,0.0,...,0.000000,1.986090e+07,2006.000000,3.000000,6.000000,1.000000,4.000000,8.000000,12.000000,16.000000
50%,12745.500000,167.500000,79.000000,103.000000,11.000000,2.000000,256.000000,256.000000,14.000000,0.0,...,0.000000,1.990030e+07,2006.000000,7.000000,15.000000,20.000000,5.000000,9.000000,13.000000,19.000000
75%,14745.500000,216.000000,97.000000,138.000000,17.000000,3.000000,256.000000,256.000000,15.000000,0.0,...,0.000000,1.993010e+07,2007.000000,10.000000,22.000000,24.000000,6.000000,10.000000,15.000000,21.000000
max,23465.000000,504.000000,204.000000,312.000000,33.000000,4.000000,512.000000,512.000000,28.000000,0.0,...,2635.000000,2.008010e+07,2008.000000,12.000000,31.000000,34.000000,18.000000,22.000000,25.000000,30.000000


In [4]:
print(df[['patient', 'PatientID']][df.PatientID != df.patient])
# extract the files for all phases
df_phases = df[['patient','x_path', 'phase']]
df_phases

      patient PatientID
5    04NEJQUZ  04NEJQU7
6    04NEJQUZ  04NEJQU7
7    04NEJQUZ  04NEJQU7
8    04NEJQUZ  04NEJQU7
9    04NEJQUZ  04NEJQU7
695  GZH20326  6ZH20326
696  GZH20326  6ZH20326
697  GZH20326  6ZH20326
698  GZH20326  6ZH20326
699  GZH20326  6ZH20326
945  PEXM9GEK  PEXM9GE 
946  PEXM9GEK  PEXM9GE 
947  PEXM9GEK  PEXM9GE 
948  PEXM9GEK  PEXM9GE 
949  PEXM9GEK  PEXM9GE 


,patient,x_path,phase
0,02Z1L14N,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,MS
1,02Z1L14N,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,ES
2,02Z1L14N,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,PF
3,02Z1L14N,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,MD
4,02Z1L14N,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,ED
...,...,...,...
1375,ZZQ0001Q,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,ED
1376,ZZQ0001Q,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,MS
1377,ZZQ0001Q,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,ES
1378,ZZQ0001Q,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,PF


In [5]:
# zip the phase files in the following structure:
#[[patient1_phase1, patient1_phase2, patient1_phase3, patient1_phase4, patient1_phase5],[patient2_phase1...] ...]

ed = df_phases[df_phases['phase'] == 'ED']['x_path']
ms = df_phases[df_phases['phase'] == 'MS']['x_path']
es = df_phases[df_phases['phase'] == 'ES']['x_path']
pf = df_phases[df_phases['phase'] == 'PF']['x_path']
md = df_phases[df_phases['phase'] == 'MD']['x_path']

# zip the phase liss into one list of lists/tuples
inputs = list(zip(ed,ms,es,pf,md))
outputs = list(zip(ms,es,pf,md,ed))

In [7]:
# initialise a generator with these files
# logging.getLogger().setLevel(logging.INFO)
from src.data.Generators import MotionDataGenerator
config['AUGMENT'] = True
config['RESAMPLE'] = True
batch_generator = MotionDataGenerator(inputs, outputs, config=config)
val_config = config.copy()
val_config['AUGMENT'] = False
validation_generator = MotionDataGenerator(inputs, outputs , config=val_config)

2021-02-05 15:37:58,800 INFO Create DataGenerator
2021-02-05 15:37:58,806 INFO Datagenerator created with: 
 shape: [40, 64, 64]
 spacing: [4, 4, 4]
 batchsize: 1
 Scaler: MinMax
 Images: 276 
 Augment: True 
 Thread workers: 1
2021-02-05 15:37:58,807 INFO Data will be augmented (shift,scale and rotate) with albumentation
2021-02-05 15:37:58,808 INFO Create DataGenerator
2021-02-05 15:37:58,815 INFO Datagenerator created with: 
 shape: [40, 64, 64]
 spacing: [4, 4, 4]
 batchsize: 1
 Scaler: MinMax
 Images: 276 
 Augment: False 
 Thread workers: 1
2021-02-05 15:37:58,816 INFO No augmentation


In [8]:
# visualise one batch
@interact
def select_image_in_batch(batch = (0,len(batch_generator), 1), im = (0,BATCHSIZE- 1, 1), slice_n=(1,11), save=False, filepath='data/temp/', filename='temp_x.npy'):
    
    global inputs, outputs
    inputs, outputs = batch_generator.__getitem__(batch)
    print(inputs.shape, outputs.shape)
    print('selected batch : ' + str(batch))
    
    selected_input = inputs[im]
    
    list(map(lambda x: show_2D_or_3D(img=x, interpol='bilinear',dpi=100,f_size=(5,5)), selected_input))
    
    """
        show_2D_or_3D(x[im], y[im], interpol='bilinear',dpi=100,f_size=(5,5))
    plt.show()
    
    plt.hist(x[im].flatten())
    plt.show()
    if save:
        ensure_dir(filepath)
        np.save(os.path.join(filepath, filename), x[im])
        logging.info('saved to {}'.format(os.path.join(filepath, filename)))
    
    """


interactive(children=(IntSlider(value=138, description='batch', max=276), IntSlider(value=0, description='im',…